In [6]:
# %pip install wget
# %pip install pyunpack

In [7]:
import os
import pandas as pd
import numpy as np
import wget
import shutil
from pyunpack import Archive

# General functions for data downloading & aggregation.
def download_from_url(url, output_path):
    """Downloads a file from a URL."""
    print('Pulling data from {} to {}'.format(url, output_path))
    wget.download(url, output_path)
    print('\nDownload complete.')

def unzip(zip_path, output_file, data_folder):
    """Unzips files and checks successful completion."""
    print('Unzipping file: {}'.format(zip_path))
    Archive(zip_path).extractall(data_folder)

    # Check if unzip was successful
    if not os.path.exists(output_file):
        raise ValueError('Error in unzipping process! {} not found.'.format(output_file))

def download_and_unzip(url, zip_path, csv_path, data_folder):
    """Downloads and unzips an online CSV file."""
    download_from_url(url, zip_path)
    unzip(zip_path, csv_path, data_folder)
    print('File successfully unzipped.')

def download_electricity():
    """Downloads electricity dataset, processes it, and saves to CSV."""
    # Define local paths
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00321/LD2011_2014.txt.zip'
    data_folder = './data'
    os.makedirs(data_folder, exist_ok=True)
    
    zip_path = os.path.join(data_folder, 'LD2011_2014.txt.zip')
    csv_path = os.path.join(data_folder, 'LD2011_2014.txt')
    output_csv = os.path.join(data_folder, 'processed_electricity_data.csv')
    
    # Download and unzip
    download_and_unzip(url, zip_path, csv_path, data_folder)
    
    print('Aggregating to hourly data...')
    df = pd.read_csv(csv_path, index_col=0, sep=';', decimal=',')
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)
    
    # Resample to hourly data
    output = df.resample('1h').mean().replace(0., np.nan)
    earliest_time = output.index.min()
    
    # Process each column in the dataset
    df_list = []
    for label in output:
        print('Processing {}'.format(label))
        srs = output[label]
        
        start_date = min(srs.fillna(method='ffill').dropna().index)
        end_date = max(srs.fillna(method='bfill').dropna().index)
        
        active_range = (srs.index >= start_date) & (srs.index <= end_date)
        srs = srs[active_range].fillna(0.)
        
        tmp = pd.DataFrame({'power_usage': srs})
        date = tmp.index
        tmp['t'] = (date - earliest_time).seconds / 60 / 60 + (
            date - earliest_time).days * 24
        tmp['days_from_start'] = (date - earliest_time).days
        tmp['categorical_id'] = label
        tmp['date'] = date
        tmp['id'] = label
        tmp['hour'] = date.hour
        tmp['day'] = date.day
        tmp['day_of_week'] = date.dayofweek
        tmp['month'] = date.month
        
        df_list.append(tmp)
    
    # Combine and finalize output
    output = pd.concat(df_list, axis=0, join='outer').reset_index(drop=True)
    output['categorical_id'] = output['id'].copy()
    output['hours_from_start'] = output['t']
    output['categorical_day_of_week'] = output['day_of_week'].copy()
    output['categorical_hour'] = output['hour'].copy()
    
    # Filter rows based on range used in academic papers
    output = output[(output['days_from_start'] >= 1096)
                    & (output['days_from_start'] < 1346)].copy()
    
    # Save final processed data
    output.to_csv(output_csv, index=False)
    print('Processed data saved to {}'.format(output_csv))
    print('Done.')

In [8]:
# Run the function
if __name__ == '__main__':
    download_electricity()


Pulling data from https://archive.ics.uci.edu/ml/machine-learning-databases/00321/LD2011_2014.txt.zip to ./data/LD2011_2014.txt.zip

Download complete.
Unzipping file: ./data/LD2011_2014.txt.zip
File successfully unzipped.
Aggregating to hourly data...
Processing MT_001
Processing MT_002
Processing MT_003


/tmp/ipykernel_54851/1030994316.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  start_date = min(srs.fillna(method='ffill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  end_date = max(srs.fillna(method='bfill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  start_date = min(srs.fillna(method='ffill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  end_date = max(srs.fillna(method='bfill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:59: FutureWarning: Series.fillna 

Processing MT_004
Processing MT_005
Processing MT_006


/tmp/ipykernel_54851/1030994316.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  start_date = min(srs.fillna(method='ffill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  end_date = max(srs.fillna(method='bfill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  start_date = min(srs.fillna(method='ffill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  end_date = max(srs.fillna(method='bfill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:59: FutureWarning: Series.fillna 

Processing MT_007
Processing MT_008
Processing MT_009


...

Processing MT_365
Processing MT_366
Processing MT_367


/tmp/ipykernel_54851/1030994316.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  start_date = min(srs.fillna(method='ffill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  end_date = max(srs.fillna(method='bfill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  start_date = min(srs.fillna(method='ffill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  end_date = max(srs.fillna(method='bfill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:59: FutureWarning: Series.fillna 

Processing MT_368
Processing MT_369
Processing MT_370


/tmp/ipykernel_54851/1030994316.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  start_date = min(srs.fillna(method='ffill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  end_date = max(srs.fillna(method='bfill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:59: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  start_date = min(srs.fillna(method='ffill').dropna().index)
/tmp/ipykernel_54851/1030994316.py:60: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  end_date = max(srs.fillna(method='bfill').dropna().index)


Processed data saved to ./data/processed_electricity_data.csv
Done.
